# Salesforce/instructblip-vicuna-7b

In [ ]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
import requests

In [ ]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", torch_dtype=torch.float16, low_cpu_mem_usage=True)
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "What is unusual about this image?"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=1,
)
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

# Brightness-Count

In [ ]:
import torch
from PIL import Image
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from collections import defaultdict
import pandas as pd
import re

# Load the InstructBLIP model and processor
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-vicuna-7b",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
).to("cuda:0")

# Load the dataset
data = pd.read_csv("DARE Dataset/only noisy dataset [based on category aka trick/vcr..]/CountCategoryData.csv")

# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Brightness/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Contrast-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Contrast/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Defocus-blur-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Defocus-blur/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Elastic-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Elastic/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Fog-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Fog/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Frost-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Frost/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Gaussian-noise-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Gaussian-noise/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# Impulse-Noise-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Impulse-noise/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")

# JPEG-compresion-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/JPEG-compression/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Pixelate-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Pixelate/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Rain-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Rain/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Saturation-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Saturation/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Shot-noise-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Shot-noise/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Snow-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Snow/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Spatter-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Spatter/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Speckle-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Speckle-noise/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


# Zoom-blur-Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Zoom-Blur/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")


## Motion-blur Count

In [ ]:
# Initialize counters for accuracy per noise type
noise_type_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})

# Iterate over rows where category is "count"
for index, row in data.iterrows():
    if row["category"] != "count":
        continue  # Skip non-count categories

    # Load the corresponding image
    image_path = f"Noisy DARE TEST/Motion-blur/{row['path']}"
    image = Image.open(image_path)

    # Prepare the prompt template
    question_type = row["category"]
    question = row["modified_question"]
    options = {
        "A": row["A"],
        "B": row["B"],
        "C": row["C"],
        "D": row["D"]
    }

    prompt_text = (
        f"The following are multiple choice questions about {question_type}. "
        "You should directly answer the question by choosing the correct option given the image and the question. "
        "Give only the letter indicating the correct answer e.g. 'A'\n"
        f"Question: {question}\n"
        "Options:\n"
        f"A. {options['A']}\n"
        f"B. {options['B']}\n"
        f"C. {options['C']}\n"
        f"D. {options['D']}\n"
        "Answer:"
    )

    # Prepare inputs
    inputs = processor(images=image, text=prompt_text, return_tensors="pt").to("cuda:0")

    # Generate model output
    output = model.generate(**inputs, max_new_tokens=1000)
    full_answer = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    print(f"Full model output for Question ID {row['id']}: {full_answer}")
    # print(f"Full model output for Question ID: {full_answer}")
    
    # Extract the predicted answer (option letter)
    if match := re.search(r"Answer:\s*([A-D])", full_answer):
        predicted_answer = match.group(1)  # Extract the letter (A, B, C, or D)
    else:
        print(f"Could not extract answer for Question ID {row['id']}: {full_answer}")
        predicted_answer = "Unknown"
    
    predicted_answer = [predicted_answer]

    # Get the noise type
    noise_type = row["modified_question_function_name"]
    actual_answer = [row["answer"][2]]

    # Compare with the actual answer and update counters
    if predicted_answer == actual_answer:
        noise_type_accuracy[noise_type]["correct"] += 1
    noise_type_accuracy[noise_type]["total"] += 1

    # Print the prediction
    print(f"Question ID {row['id']} - Predicted Answer: {predicted_answer} - Actual Answer: {row['answer']} - Noise Type: {noise_type}")

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0
    print(f"Noise Type: {noise_type} - Accuracy: {accuracy:.2%}")

In [ ]:
# Initialize counters for overall accuracy
total_correct = 0
total_predictions = 0

# Calculate and print accuracy per noise type
for noise_type, counts in noise_type_accuracy.items():
    correct = counts["correct"]
    total = counts["total"]
    print(f"Noise Type : {noise_type}, Correct Predictions: {correct}, Total: {total}")
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy for noise type '{noise_type}': {accuracy:.2f}%")
    
    # Update overall counters
    total_correct += correct
    total_predictions += total

# Calculate overall accuracy
print(f"Total Correct: {total_correct}, Total Predictions: {total_predictions}")
overall_accuracy = (total_correct / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")